Input and output available in 'SQLite3-Problem1.xlsx'

In [13]:
import pandas as pd
import sqlite3 as db

#Connect to SQLite DB named 'sqliteprac.db'. If it's not prsent, it will create a new one.
conn = db.connect('sqliteprac.db')

In [2]:
#Create a cursor to communicate with the datbase
cn = conn.cursor()

In [4]:
#Create a table and provide column details
cn.execute( "create table icc_world_cup(Team_1 Varchar(20),Team_2 Varchar(20),Winner Varchar(20))")

In [5]:
icc_values = [('India','SL','India'),('SL','Aus','Aus'),('SA','Eng','Eng'),('Eng','NZ','NZ'),('Aus','India','India')]

In [6]:
query= "insert into icc_world_cup VALUES(?,?,?)"

In [7]:
#Insert values into the table
cn.executemany(query,icc_values)

In [8]:
#Query to read values in the table
cn.execute("select * from icc_world_cup")
print(cn.fetchall())

[('India', 'SL', 'India'), ('SL', 'Aus', 'Aus'), ('SA', 'Eng', 'Eng'), ('Eng', 'NZ', 'NZ'), ('Aus', 'India', 'India')]


In [9]:
#Query to read values in the table in Pandas
query = 'select * from icc_world_cup'
pd.read_sql_query(query,conn)

,Team_1,Team_2,Winner
0,India,SL,India
1,SL,Aus,Aus
2,SA,Eng,Eng
3,Eng,NZ,NZ
4,Aus,India,India


In [10]:
#Query to create a new column named win_flag and post the value 1 for the winning teams in every row 
#Query done individually for columns Team_1 and Team_2 and then union of both the columns
query = """
select Team_1 as Team_name, 
       case when Team_1 = Winner then 1 else 0 end as win_flag
from icc_world_cup
union all
select Team_2 as Team_name, 
       case when Team_2 = Winner then 1 else 0 end as win_flag
from icc_world_cup
"""
pd.read_sql_query(query,conn)

,Team_name,win_flag
0,India,1
1,SL,0
2,SA,0
3,Eng,0
4,Aus,0
5,SL,0
6,Aus,1
7,Eng,1
8,NZ,1
9,India,1


In [20]:
#Query to determine the matches lost using the win_flag, count
query = """
with win as
(
select Team_1 as Team_name, 
       case when Team_1 = Winner then 1 else 0 end as win_flag
from icc_world_cup
union all
select Team_2 as Team_name, 
       case when Team_2 = Winner then 1 else 0 end as win_flag
from icc_world_cup
)

select Team_name, 
       count(1) as 'No_of_matches_played', 
       sum(win_flag) as 'No_of_matches_won',
       count(1) - sum(win_flag) as 'No_of_matches_lost'
from win
group by Team_name
order by No_of_matches_won desc
"""
pd.read_sql_query(query,conn)

,Team_name,No_of_matches_played,No_of_matches_won,No_of_matches_lost
0,India,2,2,0
1,NZ,1,1,0
2,Eng,2,1,1
3,Aus,2,1,1
4,SL,2,0,2
5,SA,1,0,1


In [11]:
#Commit the changes in the DB
conn.commit()

In [12]:
#Close the connection to the DB
conn.close()